In [4]:
# Vector Store
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm =ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter =CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size= 600,
    chunk_overlap= 100,
)
loader = UnstructuredFileLoader("./RAG/1984.docx")

docs = loader.load_and_split(text_splitter= splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human","{question}"),
])

chain = {"context" : retriever, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a building from which you can see the four Ministries of Oceania simultaneously. It is described as having a roof from which you can view all four Ministries, including the Ministry of Truth, Ministry of Peace, Ministry of Love, and Ministry of Plenty. The Ministries are the main government buildings in Oceania.')